# Sudoku Solver Using Constraint Satisfaction Problem (CSP)

In this notebook, we will solve Sudoku puzzles using techniques from Constraint Satisfaction Problems (CSP) such as Naked Single, Hidden Single, and algorithms like AC-3, MRV (Minimum Remaining Values), and Degree Heuristic.

## What is CSP?
A Constraint Satisfaction Problem is a mathematical problem where we must assign values to a set of variables in such a way that the constraints between them are satisfied. Sudoku is a classic example of a CSP, where each cell is a variable and the constraints are the rules of the game (each row, column, and 3x3 box must contain the digits 1-9 without repetition).

## Import libraries

In [1]:
import os
import itertools as it
from collections import deque

## Function: `read_board`
This function reads the Sudoku board from a file, ensuring it contains exactly 81 lines (for the 81 cells). Each cell is either a known value or a set of possible values.

In [2]:
def read_board(file_path):
    """Read the text file and create the board dictionary."""
    board = {}
    with open(file_path, 'r') as file:
        lines = file.readlines()
        if len(lines) != 81:
            raise ValueError("The file must contain exactly 81 lines.")
        rows = 'ABCDEFGHI'
        columns = '123456789'
        for i, line in enumerate(lines):
            row = rows[i // 9]
            column = columns[i % 9]
            cell = row + column
            options = line.strip()
            board[cell] = set(options) if len(options) > 1 else {options}
    return board

def print_board(board):
    """Print the Sudoku board."""
    rows = 'ABCDEFGHI'
    columns = '123456789'
    for row in rows:
        for column in columns:
            cell = row + column
            print(''.join(board[cell]) if len(board[cell]) == 1 else '.', end=' ')
        print()

### Function: `naked_single`

The **Naked Single** technique simplifies the Sudoku board by checking if any cell contains only one possible value (a "naked single"). If such a cell is found, that value is removed from the possible values of the other cells in the same row, column, or box.

This helps reduce the possible values for neighboring cells and moves us closer to the solution.


In [3]:
def naked_single(board, constraints):
    """
    Applies the Naked Single technique to the board.

    Parameters:
    board (dict): The current Sudoku board.
    constraints (list): A list of constraints (rows, columns, boxes).

    Returns:
    None: The board is modified in place.
    """
    for const in constraints:
        for KeyVar in const:
            if len(board[KeyVar]) == 1:  # If there's only one possible value
                for KeyXDelete in const:
                    if KeyXDelete != KeyVar:
                        # Remove that value from the other cells in the same group
                        board[KeyXDelete].discard(next(iter(board[KeyVar])))

### Function: `hidden_single`

This function implements the Hidden Single technique. It goes through each group of constraints (row, column, or box) and checks if a digit can only be placed in one specific cell. If such a cell is found, it assigns that digit to the cell.


In [4]:
def hidden_single(board, constraints):
    """
    Applies the Hidden Single technique to the Sudoku board.

    Parameters:
    board (dict): The current Sudoku board.
    constraints (list): A list of constraints (rows, columns, boxes).

    Returns:
    None: The board is modified in place.
    """
    for const in constraints:
        for digit in '123456789':
            count = 0
            last_key = None
            for KeyVar in const:
                # Check if the digit is a possible value for this cell
                if digit in board[KeyVar]:
                    count += 1
                    last_key = KeyVar
            # If the digit can only be placed in one cell, it must go there
            if count == 1:
                board[last_key] = {digit}

### Algorithm: `AC-3` (Arc Consistency 3)

AC-3 is a fundamental algorithm used to enforce consistency between variables in CSPs. In Sudoku, it helps by reducing the possible values for each cell by checking pairs of neighboring cells. If a cell has a value that contradicts a neighbor, that value is removed.

This is especially useful for eliminating impossible values early in the search process.


In [5]:
def ac3(board, constraints):
    """
    Implements the AC-3 algorithm to reduce the domain of variables in the Sudoku board.

    Parameters:
    board (dict): The current Sudoku board.
    constraints (list): A list of constraints (rows, columns, boxes).

    Returns:
    bool: Returns True if the board is arc-consistent, otherwise False.
    """
    queue = deque([(Xi, Xj) for const in constraints for Xi in const for Xj in const if Xi != Xj])
    
    while queue:
        Xi, Xj = queue.popleft()
        if revise_arc_consistency(Xi, Xj, board):
            if len(board[Xi]) == 0:
                return False
            # We change the set of lists to simply iterate over the constraints
            for const in constraints:
                if Xi in const:
                    for Xk in const:
                        if Xk != Xi and Xk != Xj:
                            queue.append((Xk, Xi))
    return True

def revise_arc_consistency(Xi, Xj, board):
    """Check if Xi's domain can be reduced by removing inconsistent values with Xj."""
    removed = False
    for x in board[Xi].copy():
        if not any(x != y for y in board[Xj]):
            board[Xi].discard(x)
            removed = True
    return removed

### Function: `mrv`

The MRV (Minimum Remaining Values) heuristic is used to select the variable (Sudoku cell) with the fewest possible values remaining. This heuristic is based on the idea that the most constrained variable should be solved first, as it limits future choices and simplifies the problem.

For Sudoku, this translates into choosing the cell with the fewest possible digits to assign.


In [6]:
def mrv(board):
    """
    MRV heuristic (Minimum Remaining Values): Selects the cell with the fewest possible values.

    Parameters:
    board (dict): The current Sudoku board.

    Returns:
    str: The cell (key) with the minimum remaining values.
    """
    # Return the cell with the minimum possible values, ignoring cells already solved (with 1 value)
    return min((v for v in board if len(board[v]) > 1), key=lambda x: len(board[x]), default=None)

### Function: `degree_heuristic`

The Degree Heuristic is used when multiple variables have the same number of remaining values. It breaks the tie by selecting the variable involved in the largest number of constraints (i.e., the cell that affects the most neighboring cells). 

This helps in making the most restrictive decision, further reducing the search space.


In [7]:
def degree_heuristic(board, constraints):
    """
    Degree heuristic: Selects the cell that participates in the most constraints.

    Parameters:
    board (dict): The current Sudoku board.
    constraints (list): The list of constraints (rows, columns, boxes).

    Returns:
    str: The cell (key) involved in the most constraints.
    """
    return max((v for v in board if len(board[v]) > 1), key=lambda x: sum(1 for c in constraints if x in c), default=None)

### Function: `solve`

The `solve` function uses backtracking to find a solution to the Sudoku puzzle. It employs both the MRV heuristic and the Degree Heuristic to guide the search process, reducing the search space and making the solving process more efficient.


In [8]:
def is_board_complete(board):
    """Check if the board is complete."""
    return all(len(board[var]) == 1 for var in board)

def solve(board, constraints, verbose=False):
    """
    Solves the Sudoku puzzle using backtracking, MRV, and Degree Heuristic.

    Parameters:
    board (dict): The current Sudoku board.
    constraints (list): The list of constraints (rows, columns, boxes).
    verbose (bool): If True, prints the steps taken by the solver.

    Returns:
    dict: The solved board, or None if no solution is found.
    """
    # Check if the board is fully solved
    if is_board_complete(board):
        return board
    
    # Select the variable to assign a value using MRV, or Degree Heuristic as a tie breaker
    var = mrv(board) or degree_heuristic(board, constraints)
    
    if not var:
        return None

    # Try assigning each possible value to the selected variable
    for value in board[var].copy():
        # Create a copy of the board to test the assignment
        board_copy = {v: board[v].copy() for v in board}
        board_copy[var] = {value}

        # If the assignment is valid (AC3 holds), continue searching
        if ac3(board_copy, constraints):
            if verbose:
                print(f"Assigning {value} to variable {var}")
            solution = solve(board_copy, constraints, verbose)
            if solution:
                return solution
    
    return None

### Example: Solving a Sudoku Puzzle

Let's now test the full CSP solver on a difficult Sudoku puzzle.
#### relative path


In [9]:

def generate_constraints():
    """Generate the constraints for Sudoku."""
    rows = 'ABCDEFGHI'
    columns = '123456789'
    
    def group(boxes):
        return [tuple(box) for box in boxes]

    # Rows
    row_groups = group([[row + column for column in columns] for row in rows])
    # Columns
    column_groups = group([[row + column for row in rows] for column in columns])
    # 3x3 Boxes
    box_groups = group([[row + column for row in rows[i:i+3] for column in columns[j:j+3]] for i in range(0, 9, 3) for j in range(0, 9, 3)])
    
    return row_groups + column_groups + box_groups

def main():
    default_path = r'Board_impossible_SD9BJKIA.txt'
    file_path = input("Enter the path of the Sudoku file (or press Enter to use the default path): ")
    if not file_path:
        file_path = default_path

    if not os.path.exists(file_path):
        print(f"The file {file_path} does not exist.")
        return

    board = read_board(file_path)
    constraints = generate_constraints()
    
    print("Initial board:")
    print_board(board)
    
    solution = solve(board, constraints, verbose=True)
    
    if solution:
        print("\nSolution found!")
        print_board(solution)
    else:
        print("\nNo solution found.")

if __name__ == "__main__":
    main()


Initial board:
. 6 . . 5 . . . 4 
. . 9 . . . . . . 
7 1 5 . . . . . . 
. . 3 . 1 . . . 5 
. . . 7 . 3 2 . . 
8 . 1 . 4 5 . 9 . 
. . 7 . . . 9 . . 
2 . 8 . . . 5 4 . 
6 . . 9 2 . 7 . . 
Assigning 3 to variable A1
Assigning 1 to variable A4
Assigning 6 to variable C7
Assigning 2 to variable B9
Assigning 3 to variable B9
Assigning 8 to variable A4
Assigning 6 to variable B7
Assigning 7 to variable B5
Assigning 2 to variable B6
Assigning 1 to variable B6
Assigning 3 to variable B5

Solution found!
3 6 2 8 5 9 1 7 4 
4 8 9 1 3 7 6 5 2 
7 1 5 4 6 2 8 3 9 
9 7 3 2 1 8 4 6 5 
5 4 6 7 9 3 2 1 8 
8 2 1 6 4 5 3 9 7 
1 3 7 5 8 4 9 2 6 
2 9 8 3 7 6 5 4 1 
6 5 4 9 2 1 7 8 3 
